In [6]:
import os
import sqlite3
import pandas as pd

# Path to the main folder containing subfolders with databases
main_folder = 'data/original_dev/dev_databases'

# List all subfolders in the main folder
databases = [d for d in os.listdir(main_folder) if os.path.isdir(os.path.join(main_folder, d))]

# Initialize a list to store the results
results = []

# Loop through each database subfolder
for db_name in databases:
    db_path = os.path.join(main_folder, db_name, f"{db_name}.sqlite")
    
    # Check if the SQLite file exists
    if os.path.exists(db_path):
        # Connect to the SQLite database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Get the list of tables
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        num_tables = len(tables)
        
        # Initialize the count of columns
        num_columns = 0
        
        # Loop through each table to count columns
        for table in tables:
            table_name = table[0]
            cursor.execute(f"PRAGMA table_info(\"{table_name}\");")
            columns = cursor.fetchall()
            num_columns += len(columns)
        
        # Add the results to the list
        results.append({
            'Database': db_name,
            'Number of Tables': num_tables,
            'Number of Columns': num_columns
        })
        
        # Close the connection
        conn.close()

# Convert the results to a DataFrame and display it
df_results = pd.DataFrame(results)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df_results)

                   Database  Number of Tables  Number of Columns
0   debit_card_specializing                 6                 23
1                 financial                 8                 55
2                 formula_1                14                 96
3        california_schools                 3                 89
4                card_games                 7                117
5       european_football_2                 8                201
6     thrombosis_prediction                 3                 64
7                toxicology                 4                 11
8              student_club                 8                 48
9                 superhero                10                 31
10       codebase_community                 8                 71


In [8]:

# List all subfolders in the main folder
databases = [d for d in os.listdir(main_folder) if os.path.isdir(os.path.join(main_folder, d))]

# Initialize a list to store the results
results = []

# Loop through each database subfolder
for db_name in databases:
    db_path = os.path.join(main_folder, db_name, f"{db_name}.sqlite")
    
    # Check if the SQLite file exists
    if os.path.exists(db_path):
        # Connect to the SQLite database
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Get the list of tables, excluding 'sqlite_sequence'
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name != 'sqlite_sequence';")
        tables = cursor.fetchall()
        num_tables = len(tables)
        
        # Initialize the count of columns
        num_columns = 0
        
        # Loop through each table to count columns
        for table in tables:
            table_name = table[0]
            cursor.execute(f"PRAGMA table_info(\"{table_name}\");")
            columns = cursor.fetchall()
            num_columns += len(columns)
        
        # Add the results to the list
        results.append({
            'Database': db_name,
            'Number of Tables': num_tables,
            'Number of Columns': num_columns
        })
        
        # Close the connection
        conn.close()

# Convert the results to a DataFrame
df = pd.DataFrame(results)

# Calculate totals
total_tables = df['Number of Tables'].sum()
total_columns = df['Number of Columns'].sum()

# Append the totals row to the DataFrame
totals_row = pd.DataFrame([['Total', total_tables, total_columns]], columns=df.columns)
df_with_totals = pd.concat([df, totals_row], ignore_index=True)

# Convert to LaTeX table with totals
latex_table_with_totals = df_with_totals.to_latex(index=False)

# Print LaTeX table
print(latex_table_with_totals)

\begin{tabular}{lrr}
\toprule
Database & Number of Tables & Number of Columns \\
\midrule
debit_card_specializing & 5 & 21 \\
financial & 8 & 55 \\
formula_1 & 13 & 94 \\
california_schools & 3 & 89 \\
card_games & 6 & 115 \\
european_football_2 & 7 & 199 \\
thrombosis_prediction & 3 & 64 \\
toxicology & 4 & 11 \\
student_club & 8 & 48 \\
superhero & 10 & 31 \\
codebase_community & 8 & 71 \\
Total & 75 & 798 \\
\bottomrule
\end{tabular}



In [16]:
import pandas as pd

# Path to the Excel file
file_path = 'annotations/Difficulty Annotation.xlsx'

# Read the specific sheet from the Excel file
df = pd.read_excel(file_path, sheet_name='Sammanställning')

# Group by database_name and Final difficulty, and count occurrences
difficulty_counts = df.groupby(['database_name', 'Final difficulty']).size().unstack(fill_value=0)



difficulty_counts


Final difficulty,Easy,Hard,Medium,Very Hard
database_name,,,,
california_schools,24,30,32,3
card_games,37,50,28,0
codebase_community,62,5,4,0
debit_card_specializing,17,1,3,0
european_football_2,34,103,62,0
financial,21,11,9,13
formula_1,49,10,35,0
student_club,39,0,9,0
superhero,30,0,1,0


In [36]:
import pandas as pd

# Path to the Excel file
file_path = 'annotations/Original_BIRD_Annotations2.xlsx'

# Read the sheets for each annotator
df_axel = pd.read_excel(file_path, sheet_name='Axel')
df_erik = pd.read_excel(file_path, sheet_name='Erik')

# Define all quality classes
quality_classes = ["No description", "I can't tell", "Incorrect", "Somewhat correct", "Almost Perfect", "Perfect"]

# Calculate the percentage of each quality class for Axel
axel_counts = df_axel['Annotation'].value_counts(normalize=True) * 100
axel_counts = axel_counts.reindex(quality_classes).fillna(0)

# Calculate the percentage of each quality class for Erik
erik_counts = df_erik['Annotation'].value_counts(normalize=True) * 100
erik_counts = erik_counts.reindex(quality_classes).fillna(0)

# Combine the counts into a single DataFrame
combined_counts = pd.DataFrame({
    'Annotator 1': axel_counts,
    'Annotator 2': erik_counts
})


# Calculate the mean percentage for each quality class
combined_counts['Mean Percentage'] = combined_counts.mean(axis=1)


combined_counts


,Annotator 1,Annotator 2,Mean Percentage
Annotation,,,
No description,18.272841,18.272841,18.272841
I can't tell,1.501877,0.125156,0.813517
Incorrect,2.628285,1.126408,1.877347
Somewhat correct,40.801001,36.795995,38.798498
Almost Perfect,0.000000,0.000000,0.000000
Perfect,36.795995,43.679599,40.237797


In [36]:
import pandas as pd

# Load the Excel file
file_path = 'annotations/LLM Prediction Annotations(11).xlsx'
sheets = ['Forsberg', 'Oscar']

# Read the sheets into separate dataframes
df_hjalmar = pd.read_excel(file_path, sheet_name=sheets[0])
df_sorme = pd.read_excel(file_path, sheet_name=sheets[1])

# Extract the annotation columns
annotations_hjalmar = df_hjalmar['Annotation LLM 3']
annotations_sorme = df_sorme['Annotation LLM 3']

# Calculate the mean number of annotations in each class
mean_annotations_hjalmar = annotations_hjalmar.value_counts(normalize=True)
mean_annotations_sorme = annotations_sorme.value_counts(normalize=True)

# Combine the means into a single dataframe
mean_annotations = pd.concat([mean_annotations_hjalmar, mean_annotations_sorme], axis=1)
mean_annotations.columns = ['Forsberg', 'Oscar']

# Calculate the overall percentage between the classes
overall_percentage = mean_annotations.mean(axis=1) * 100

# Display the results
mean_annotations, overall_percentage


(                     Forsberg     Oscar
 Annotation LLM 3                       
 4. Perfect           0.775689  0.723058
 No description       0.068922  0.068922
 3. Almost perfect    0.062657  0.100251
 2. Somewhat correct  0.050125  0.072682
 1. Incorrect         0.042607  0.035088,
 Annotation LLM 3
 4. Perfect             74.937343
 No description          6.892231
 3. Almost perfect       8.145363
 2. Somewhat correct     6.140351
 1. Incorrect            3.884712
 dtype: float64)

In [37]:
import pandas as pd

# Load the CSV file
csv_file_path = "output/GOLD_DATASET_FINAL.csv"
csv_data = pd.read_csv(csv_file_path)

# Load the Excel file
excel_file_path = "annotations/Difficulty Annotation.xlsx"
excel_data = pd.read_excel(excel_file_path, sheet_name="Sammanställning")

# Append the 'Final difficulty' column to the CSV data
csv_data['Final difficulty'] = excel_data['Final difficulty']

# Save the new dataframe to a new CSV file
output_file_path = "output/GOLD_DATASET_FINAL_with_difficulties.csv"
csv_data.to_csv(output_file_path, index=False)

print(f"File saved to {output_file_path}")


File saved to output/GOLD_DATASET_FINAL_with_difficulties.csv


In [49]:
import pandas as pd

data = pd.read_excel('annotations/LLM Prediction Annotations(12).xlsx', sheet_name="Scores")

# Filter the relevant columns
data_filtered = data[['Annotation 1 LLM 2', 'Annotation 2 LLM 2', 'Final difficulty']]

# Convert the annotations to numeric values
annotation_mapping = {
    "4. Perfect": 4,
    "3. Almost perfect": 3,
    "2. Somewhat correct": 2,
    "1. Incorrect": 1
}

data_filtered['Annotation 1 LLM 1'] = data_filtered['Annotation 1 LLM 1'].map(annotation_mapping)
data_filtered['Annotation 2 LLM 1'] = data_filtered['Annotation 2 LLM 1'].map(annotation_mapping)

# Calculate the average annotation per difficulty
data_filtered['Average Quality'] = data_filtered[['Annotation 1 LLM 1', 'Annotation 2 LLM 1']].mean(axis=1)

# Group by difficulty and calculate the mean quality
difficulty_quality = data_filtered.groupby('Final difficulty')['Average Quality'].mean().reset_index()

latex_table = difficulty_quality.to_latex(index=False)

# Display the LaTeX table code
print(latex_table)

KeyError: 'Annotation 1 LLM 1'